# LionAGI introduction 3 - LLM sessions

In [1]:
import lionagi as li

In [2]:
from timeit import default_timer as timer
start = timer()

In [3]:
system = """
You are a helpful assistant. You are asked to perform as a calculator. Return as an integer.
"""

calculator = li.Session(system=system)

In [4]:
a = -3
b = 4

context = {
    "number1": a,
    "number2": b,
}

instruct1 = {
    "sum the absolute values": "provided with 2 numbers, return the sum of their absolute values",}

instruct2 = {
    "multiplication": "provided with 2 numbers, return their multiplication",}

instruct3 = {
    "case positive": "if the result from previous step is positive, times 2 to the previous step's result",
    "case negative": "elif the result from previous step is negative, plus 2 to the previous step's result",
    "case zero": "elif the result from previous step is zero, return the previous step's result",}


In [5]:
calculator.initiate(instruction=instruct1, context=context, model="gpt-3.5-turbo")

'7'

In [6]:
cal1 = calculator.initiate(instruction=instruct1, context=context, model="gpt-3.5-turbo")
cal2 = calculator.followup(instruction=instruct3, model="gpt-4", temperature=0.5)

print(f"Given {a} and {b}, the sum of absolute values is {cal1}")
print(f"Since the step 1 result is {'positive' if int(cal1)>0 else 'negative'}, the second step result is {cal2}")

Given -3 and 4, the sum of absolute values is 7
Since the step 1 result is positive, the second step result is 14


In [7]:
cal1 = calculator.initiate(instruction=instruct2, context=context, model="gpt-3.5-turbo")
cal2 = calculator.followup(instruction=instruct3, model="gpt-4", temperature=0.5, n=3)

print(f"Given {a} and {b}, the multiplication product is {cal1}")
print(f"Since the step 1 result is {'positive' if int(cal1)>0 else 'negative'}, the second step result is {cal2}")

Given -3 and 4, the multiplication product is -12
Since the step 1 result is negative, the second step result is -10


In [8]:
intruct4 = {"string to number": "return as a number by itself",}

calculator.followup(instruction=intruct4, model="gpt-4", temperature=0.5, n=3)

'-10'

In [9]:
#### ok now let's see how we can make it more interesting
import numpy as np
num_iterations = 5

ints1 = np.random.randint(-10, 10, size=num_iterations)
ints2 = np.random.randint(0, 10, size=num_iterations)
cases = np.random.randint(0,2, size=num_iterations)
# let's define a simple parser function

f = lambda i: {"number1": str(ints1[i]), "number2": str(ints2[i]), "case_": str(cases[i])}
contexts = li.l_call(range(num_iterations), f)

In [10]:
system = """
You are a helpful assistant. You are asked to perform as a calculator. Return as an integer.
"""

context = {
    "number1": a,
    "number2": b,
}

instruct1 = {
    "sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|",}

instruct2 = {
    "diff the absolute values": "provided with 2 numbers, return the difference of absolute values. i.e. |x|-|y|",}

instruct3 = {
    "if previous response is positive": "times 2. i.e. *2", # case 1
    "else": "plus 2. i.e. +2",                              # case 2
}

In [11]:
async def calculator_workflow(context_):
    calculator = li.Session(system=system, api_service=li.async_api_service)
    context = context_.copy()
    case = int(context.pop("case_"))
    
    if case == 0:
        await calculator.ainitiate(instruction=instruct1, context=context, model="gpt-4", temperature=0.5)
    elif case == 1:
        await calculator.ainitiate(instruction=instruct2, context=context, model="gpt-4", temperature=0.5)
    
    await calculator.afollowup(instruction=instruct3, model="gpt-4", temperature=0.3)
    return li.l_call(calculator.conversation.responses, lambda i: i['content'])

In [12]:
start1 = timer()

outs = await li.al_call(contexts, calculator_workflow)

elapsed_time = timer() - start1
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 5
run clock time: 3.00 seconds


In [13]:
for idx, out in enumerate(outs):
    print(f"Inputs: {ints1[idx]}, {ints2[idx]}, case: {cases[idx]}\n")
    print(f"Outputs: {out}")
    print("------\n")

Inputs: -2, 5, case: 0

Outputs: ['7', '14']
------

Inputs: 1, 9, case: 0

Outputs: ['10', '20']
------

Inputs: 0, 0, case: 0

Outputs: ['0', '2']
------

Inputs: 0, 4, case: 1

Outputs: ['-4', '-2']
------

Inputs: 3, 8, case: 0

Outputs: ['11', '22']
------



In [14]:
elapsed_time = timer() - start

In [15]:
print(f"Notebook total runtime {elapsed_time:0.2f} seconds")

Notebook total runtime 11.20 seconds
